In [ ]:
from openweatherUtils import * 

# Example usage
if __name__ == "__main__":
    # Simulated API response for demonstration purposes
    example_response = {
        "list": [
            {
                "dt_txt": "2022-08-30 16:00:00",
                "rain": {"1h": 0.13}
            },
            {
                "dt_txt": "2022-08-30 17:00:00"
                # 'rain' key is missing
            },
            {
                "dt_txt": "2022-08-30 18:00:00",
                "rain": {"1h": 0.24}
            },
            {
                "dt_txt": "2022-08-30 19:00:00",
                "rain": {}
                # '1h' key is missing within 'rain'
            }
        ]
    }

decoded_data = decode_hourly_forecast(example_response)
print(decoded_data)

    

[{'dt_txt': '2022-08-30 16:00:00', 'rain_1h': 0.13}, {'dt_txt': '2022-08-30 17:00:00', 'rain_1h': 0}, {'dt_txt': '2022-08-30 18:00:00', 'rain_1h': 0.24}, {'dt_txt': '2022-08-30 19:00:00', 'rain_1h': 0}]


In [1]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from gridUtils import *
from openweatherUtils import *
from tqdm import tqdm

# 输入路径
nc_path = "../data/results_3di.nc"
# 输出文件路径
output_file = "weather_data.json"

# 生成网格点
points = constructWeatherGrid(500, nc_path)


# 初始化存储数据
timestamps = []
points_data = []

# 定义并行任务
def process_point(point):
    lon, lat = point
    openweather_json = get_hourly_forecast(lat, lon)
    decode_data = decode_hourly_forecast_as_array(openweather_json)
    dt_txt_array = extract_dt_txt_array(openweather_json)
    return {"point": (lon, lat), "data": decode_data, "timestamps": dt_txt_array}

# 实时进度条
progress_bar = tqdm(total=len(points), desc="Processing points", unit="point")

# 使用线程池执行并保存结果
results = []

with ThreadPoolExecutor(max_workers=40) as executor:  # 调整 max_workers 为适当线程数
    future_to_point = {executor.submit(process_point, point): point for point in points}

    for future in as_completed(future_to_point):
        point = future_to_point[future]
        try:
            result = future.result()
            # 将解码后的数据存入 points_data
            points_data.append({"point": result["point"], "data": result["data"]})
            # 初始化 timestamps 仅一次
            if not timestamps:
                timestamps = result["timestamps"]
        except Exception as e:
            print(f"Error processing point {point}: {e}")
        finally:
            progress_bar.update(1)  # 更新进度条

progress_bar.close()  # 关闭进度条

# 写入文件，单独存储 timestamps 和 points_data
output = {"timestamps": timestamps, "points_data": points_data}
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output, f, indent=4)

print(f"Results have been saved to {output_file}")


#定时call weather api -> 一天3次 -> 9点 14点 18点

#确定放在哪个文件夹以及命名规则 -> 保存在cnnModel文件夹下

#命名规则 -> 以整点时间戳命名 YYYYMMDD-HHMMSS -> 20250227-100000







Processing points: 100%|██████████| 2880/2880 [00:29<00:00, 98.20point/s] 


Results have been saved to weather_data.json
